# HDFS

HDFS is het distributed file system van Hadoop dat de basis vormt voor een breed gamma van applicaties, waaronder MapReduce.
Dit framework maakt het mogelijk om niet-gespecialiseerde hardware te gebruiken om eenvoudig datacenters op te zetten of rekenclusters te beheren.
HDFS bereikt dit doel op een aantal manieren:
* Ten eerste wordt een file opgedeeld in verschillende blokken. Deze worden verdeeld over verschillende computers zodat code meerdere delen van het bestand kan gebruiken in parallel om zo de benodigde rekenkracht te verdelen over meerdere toestellen.
* Daarnaast worden de blokken ook gedupliceerd om zo fout-toleranter te zijn in het geval van een crash (hardware of software), stroomstoring, netwerkonderbreking.

Om dit te bereiken gebruikt Hadoop een Master-Slave architectuur dat bestaat uit een enkele namenode (master) en meerdere datanodes (slaves).
De namenode houdt bij hoeveel datanodes er actief zijn, welke blokken ze hebben en welke blokken bij welke file horen.
Indien er een datanode crasht gaat deze server dit detecteren en dit oplossen door de nodige blokken te kopieren van een andere datanode zodat er steeds voldoende kopies in het systeem aanwezig zijn.
Bij elke actie die uitgevoerd moet worden in het HDFS moet er steeds gevraagd worden aan de namenode welke blokken op welke datanodes we nodig hebben voor de gewenste file uit te lezen of code voor uit te voeren.
Het is dus duidelijk dat deze namenode een single-point-of-failure is wat ideaal is voor de availability van de cluster.
Dit kan opgelost worden door HDFS te runnen in een high-availability mode wat ervoor zorgt dat er steeds een backup aanwezig is voor de namenode die zeer snel de werking kan overnemen.
Deze structuur maakt het eenvoudig om aan horizontal scaling te doen door extra servers toe te voegen zonder dat er downtime is voor de hele cluster.

Dit resulteer in de volgende kenmerken van HDFS:
* High Throughput
* Scalability
* High Availability
* Data Reliability
* Fault Tolerance

## Starten en stoppen van de cluster

De cluster kan gestart worden door alle docker containers te starten die gedefinieerd worden in de yaml-file.
Dit kan via de docker desktop gui of via de command line door middel van het docker-compose commando.
Stoppen kan dan via dezelfde methodes

## Communiceren met het HDFS

Er zijn verschillende manieren om dit distributed bestandssysteem te gebruiken.
Ten eerste kan je gebruik maken van een command line interface (CLI) om bestanden uit te lezen, op te laden, ...
Daarnaast zijn er ook wrappers voor verschillende talen die toelaten om rechtstreeks vanuit code te interageren met het HDFS.
De voordelen van deze wrappers over een CLI zijn:
* Flexibiler om te interageren in applicaties.
* Gebruiksvriendelijker en gemakkelijker om te automatiseren.
* Eenvoudiger in onderhoud en te debuggen
* Volledige functionaliteit van een programmeertaal kan gebruikt worden zoals OO.

### Instantiering van een client

Veel verschillende talen beschikken over een wrapper om te communiceren met een HDFS.
In deze cursus gaan we gebruik maken van [InsecureClient in hdfscli](https://hdfscli.readthedocs.io/en/latest/quickstart.html) wrapper.
De eerste stap in het gebruik van de wrapper is te bepalen hoe de namenode van het hdfs gevonden kan worden.
Dit gebeurt als volgt:

In [4]:
from hdfs import InsecureClient

# connecteer met de namenode
client = InsecureClient("http://localhost:9870", user='bigdata') 

# 1 opmerking voor het werken met jupyter lab -> er is een rudimentaire vorm van intellisense (je kan de tab-toets gebruiken voor aan te vullen)
# je krijgt niet zoveel tips als in visual studio code om code te schrijven
# zorg dat je zeker het goed oefent, zodat je zeker de basis kent

if client.status('/bla', strict=False): # zonder False krijg je een foutmelding
    print('Path exists')
else:
    print('Path does not exist')

Path does not exist


In [6]:
# via cli
!hdfs dfs -test -d /

### Aanmaken van files en directories

Om nu bestanden en folders aan te maken op dit distributed file systeem kunnen onderstaande functies gebruikt worden.

In [9]:
if client.status('hdfs', strict=False) is None: # dit gaat het pad /user/bigdata/hdfs controleren
    client.makedirs('hdfs')
    print('directory created')

# zelfde als hierboven
!hdfs dfs -mkdir -p /user/bigdata/hdfscli

In [25]:
# make files

# client.upload(pad cluster, lokaal pad)
#client.upload('hdfs', 'davinci_notebooks.txt')
#client.upload('hdfs', 'outline_of_science.txt')
#client.upload('hdfs', 'ulysses.txt')
client.upload('hdfs/oudeboek.txt', 'ulysses.txt', overwrite =True, replication=2, blocksize=1048576)
client.write('hdfs/oudeboek2.txt', 'ulysses.txt') # kan meer zaken gaan schrijven dan files (bvb in dit geval de gewone string ulysses.txt

In [ ]:
# hadoop fs -put /path/in/linux /hdfs/path
# hadoop fs -get /hdfs/path /path/in/linux

### Bekijken van het filesysteem

In [20]:
# hdfs dfs -ls command
print(client.acl_status('hdfs'))
print(client.checksum('hdfs/oudeboek.txt')) # om de authenticiteit van het bestand te verifieren
print(client.content('hdfs/oudeboek.txt'))
print(client.list('hdfs')) # dit is de ls

print(client.walk('/')) # itereer over alle bestanden, duik ook in subfolders
for f in client.walk('/'):
    print(f)

{'entries': [], 'group': 'supergroup', 'owner': 'bigdata', 'permission': '755', 'stickyBit': False}
{'algorithm': 'MD5-of-2048MD5-of-512CRC32C', 'bytes': '0000020000000000000008006cef3df24af9181269e8039f656901c200000000', 'length': 28}
{'directoryCount': 0, 'ecPolicy': 'Replicated', 'fileCount': 1, 'length': 1586336, 'quota': -1, 'snapshotDirectoryCount': 0, 'snapshotFileCount': 0, 'snapshotLength': 0, 'snapshotSpaceConsumed': 0, 'spaceConsumed': 3172672, 'spaceQuota': -1, 'typeQuota': {}}
['davinci_notebooks.txt', 'oudeboek.txt', 'outline_of_science.txt', 'ulysses.txt']
<generator object Client.walk at 0x7f14566bfe60>
('/', ['rmstate', 'user'], [])
('/rmstate', ['FSRMStateRoot'], [])
('/rmstate/FSRMStateRoot', ['AMRMTokenSecretManagerRoot', 'ProxyCARoot', 'RMAppRoot', 'RMDTSecretManagerRoot', 'ReservationSystemRoot'], ['EpochNode', 'RMVersionNode'])
('/rmstate/FSRMStateRoot/AMRMTokenSecretManagerRoot', [], ['AMRMTokenSecretManagerNode'])
('/rmstate/FSRMStateRoot/ProxyCARoot', [], ['ca

## Uitlezen van het filesysteem

In [26]:
client.download('hdfs/davinci_notebooks.txt', 'notebooks.txt', overwrite=True)

# meer gericht gaan bekijken of het in de code gaan bewerken
with client.read('hdfs/oudeboek2.txt') as reader: # dit bestand was met write gedaan ipv met upload
    content = reader.read()
    print(content)

b'ulysses.txt'


### Aanpassen van het filesysteem

In [31]:
client.delete('hdfs/oudeboek.txt', recursive=False) # recursive -> om folders te verwijderen
# hdfs dfs -rm path
# add -r tag for deleting directory

# hadoop fs -mv oldname newname
#client.rename('hdfs/oudeboek2.txt', 'hdfs/oudeboek3.txt')

client.set_permission('hdfs/oudeboek3.txt', 777)
# hdfs dfs –setrep –w 3 /tmp/logs/file.txt
client.set_replication('hdfs/oudeboek3.txt', 4)

### Oefening

Los de volgende oefeningen op:

Schrijf een Python-script dat de informatie van een bestand in HDFS ophaalt en weergeeft. De informatie moet onder andere de grootte van het bestand, de eigenaar, de groep en de permissies bevatten.

Schrijf een Python-script dat de volledige inhoud van een tekstbestand in HDFS leest en afdrukt naar de console.

Schrijf een Python-script dat een tekstbestand van de lokale schijf naar HDFS schrijft. Het script moet de inhoud van het lokale bestand lezen en deze naar een nieuw bestand in HDFS schrijven.

Schrijf een Python-script dat de permissies van een bestand in HDFS wijzigt. Het script moet de permissies instellen op een door de gebruiker opgegeven waarde (bijvoorbeeld 755).

Schrijf een Python-script dat de huidige replicatiefactor van een bestand in HDFS controleert en weergeeft.

Schrijf een Python-script dat de blokken van een bestand in HDFS ophaalt en weergeeft. Het script moet de blokken en hun locaties weergeven.

Schrijf een Python-script dat alle bestanden in een HDFS-directory zoekt die voldoen aan een bepaald naamspatroon (bijvoorbeeld alle .txt bestanden).

Schrijf een Python-script dat nieuwe inhoud toevoegt aan een bestaand bestand in HDFS. Het script moet de nieuwe inhoud aan het einde van het bestand toevoegen.

Schrijf een Python-script dat de checksum van een bestand in HDFS ophaalt en weergeeft. Dit kan nuttig zijn om de integriteit van bestanden te controleren.

Schrijf een Python-script dat de metadata van een bestand in HDFS instelt. Het script moet bijvoorbeeld een aangepaste eigenschap toevoegen aan het bestand.


Schrijf python code dat controleert of een directory bestaat in het hdfs.
Indien nee wordt de directory aangemaakt.
Indien ja, worden alle files in de directory verwijderd om van een lege directory te starten.
Upload daarna een tekst-bestand naar keuze.